In [2]:
seed=1

In [ ]:
import os

os.listdir('..')

In [ ]:
from nasbench import api

nasbench_path = '../data/nasbench_only108.tfrecord'
nb = api.NASBench(nasbench_path)

In [ ]:
import torch
from info_nas.datasets.arch2vec_dataset import get_labeled_unlabeled_datasets

#torch.backends.cudnn.benchmark = True
device = torch.device('cuda')

# device = None otherwise the dataset is save to the cuda as a whole
labeled, unlabeled = get_labeled_unlabeled_datasets(nb, device=device, seed=seed,
                                                    train_pretrained=train_pretrained,
                                                    valid_pretrained=val_pretrained,
                                                    train_labeled_path='../data/train_long.pt',
                                                    valid_labeled_path='../data/valid_long.pt')

## Test dataset shapes

In [ ]:
for i in range(len(labeled['valid_io']['inputs'])):
    if i not in labeled['valid_io']['inputs']:
        raise ValueError()
        
    assert (sum(labeled['train_io']['inputs'] == i)) == 4

In [ ]:
[l.shape for l in labeled['train_io'].values() if not isinstance(l, int)]

In [ ]:
# 7 GB per 4000 nets
labeled['train_io']['outputs'].element_size() * 4000 * 512 / 1024 / 1024 / 1024 * 1000

In [ ]:
[l.shape for l in labeled['train_net']]

In [ ]:
unlabeled.keys()

In [ ]:
unlabeled['train'][1][0].shape, unlabeled['train'][2][0].shape

In [ ]:
len(unlabeled['train'][1])

## Test model shapes

In [ ]:
from arch2vec.extensions.get_nasbench101_model import get_arch2vec_model

model, opt = get_arch2vec_model()

In [ ]:
from arch2vec.extensions.get_nasbench101_model import get_nasbench_datasets

nb_dataset = get_nasbench_datasets('../data/nb_dataset.json', batch_size=None, seed=1)

In [ ]:
#model.train()
model.eval()

batch_adj, batch_ops = nb_dataset['train'][1][:32], nb_dataset['train'][2][:32]

mu, logvar = model._encoder(batch_ops, batch_adj)
z = model.reparameterize(mu, logvar)

In [ ]:
print(f"mu shape: {mu.shape}, logvar shape: {logvar.shape}, z shape: {z.shape}")

In [ ]:
import torch.nn as nn

m = nn.Sequential(
    nn.Flatten(),
    nn.Linear(z.shape[1] * z.shape[2], 5),
    nn.ReLU()
)
m(z).shape

In [ ]:
# test unsqueeze and channels

conv = nn.Conv2d(32, 8, 1, padding=0)
conv(mu.unsqueeze(0)).shape

In [ ]:
# repeat and concat
repeated = torch.Tensor([3]).repeat(mu.shape[0], mu.shape[1], 1)
print(repeated.shape)

torch.cat([mu, repeated], axis=-1).shape

## Extended models

In [ ]:
from arch2vec.extensions.get_nasbench101_model import get_arch2vec_model
from arch2vec.extensions.get_nasbench101_model import get_nasbench_datasets

model, opt = get_arch2vec_model(device=device)

In [ ]:
print(labeled['train']['dataset'].shape)
print(labeled['train']['inputs'].shape)
print(labeled['train']['outputs'].shape)

In [ ]:
from info_nas.models.io_model import ConcatConvModel

extended_model = ConcatConvModel(model, 128, 512).to(device)

In [ ]:
in_batch = labeled['train_io']['inputs'][:32]
out_batch = labeled['train_io']['outputs'][:32]

batch_adj, batch_ops = labeled['train_net'][0][:32], labeled['train_net'][1][:32]

In [ ]:
ops_recon, adj_recon, mu, logvar, _, outputs = extended_model(batch_ops.to(device), batch_adj.to(device), in_batch.to(device))

In [ ]:
outputs.shape

### Just some tests

In [ ]:
_,_,_,_,z = model(batch_ops.to(device), batch_adj.to(device))
z = extended_model.process_z(z)
z = z.unsqueeze(-1).unsqueeze(-1).repeat(1, 1, in_batch.shape[2], in_batch.shape[3])
z.shape

In [ ]:
torch.cat([z, in_batch], dim=1).shape

## Training

In [ ]:
labeled_2 = labeled.copy()

omax = labeled_2['train']['outputs'].max()
labeled_2['train']['outputs'] /= omax

print(labeled_2['train']['outputs'].max())

In [ ]:
import torch
from info_nas.trainer import train

if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    device = torch.device('cuda')
else:
    device = None
    
model = train(labeled_2, unlabeled, nb, checkpoint_path='.', device=device,
              use_reference_model=True, batch_len_labeled=7)

In [ ]:
model[1]

In [ ]:
len(labeled['train_io']['inputs'])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

plt.figure()
plt.imshow(labeled_2['train_io']['outputs'], cmap='hot', interpolation='nearest')
plt.show()